In [1]:
pd.options.display.max_columns = 100

In [18]:
# l1adarkfnames = io.get_current_science_fnames('l1a', 'periapse*uvdark', env='production')
l1adarkfnames = !ls /Users/klay6683/Dropbox/data/iuvs/periapse_muvdarks_v02_r01/*.fits*

In [19]:
len(l1adarkfnames)

1270

In [4]:
from ipywidgets import interact, interactive, fixed

In [5]:
from iuvs import io

/Users/klay6683/miniconda3/envs/py34/lib/python3.4/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [13]:
def f(i):
    l1a = io.L1AReader(l1adarkfnames[i])
    print('n_dims:', l1a.n_dims)
    i = 0
    if l1a.n_dims == 3:
        i = 1
    print(l1a.img.shape[i])
    print(l1a.Binning['SPABINWIDTH'].shape[1])

In [22]:
interact(f, i=(0,len(l1adarkfnames)-1));

n_dims: 3
7
9


In [7]:
l1a = io.L1AReader(l1adarkfnames[0])

In [8]:
l1a.Integration

,CASE_TEMP,DET_TEMP,ET,FOV_DEG,LYA_CENTROID,MIRROR_DEG,MIRROR_DN,TIMESTAMP,UTC
0,15950,2576,4.669038e+08,17.29248,-1,8.64624,16087,4.669037e+08,2014/291 Oct 18 11:28:44.36433UTC


In [9]:
l1a.n_dims

2

In [10]:
l1a.img.shape

(7, 256)

## Generate dataframe with filename stats

In [20]:
df = pd.DataFrame([io.ScienceFilename(str(i)).as_series() for i in l1adarkfnames])

In [21]:
df.columns

Index(['basename', 'channel', 'cycle_orbit', 'instrument', 'level', 'mission',
       'mode', 'obs_id', 'p', 'phase', 'revision', 'root', 'time', 'timestr',
       'tokens', 'version', 'version_string'],
      dtype='object')

In [22]:
df['phase'].value_counts()

periapse    1270
dtype: int64

In [24]:
from ipyparallel import Client
c = Client()

In [25]:
dview = c.direct_view()
lview = c.load_balanced_view()

In [57]:
def check_for_issues(p):
    from iuvs import exceptions, io
    d = {}
    d['fname'] = p.name
    try:
        l1a = io.L1AReader(p)
    except exceptions.DimensionsError:
        d['dims'] = False
    d['kind'] = l1a.img_header['BIN_TBL'][:3]
    d['n_unique_spabins'] = l1a.n_unique_spabins
    d['n_unique_spebins'] = l1a.n_unique_spebins
    return d

In [58]:
check_for_issues(df.p[4])

{'fname': 'mvn_iuv_l1a_periapse-orbit00110-muvdark_20141018T204249_v02_r01.fits.gz',
 'kind': 'LIN',
 'n_unique_spabins': 1,
 'n_unique_spebins': 1}

In [59]:
doing = df.p
results = lview.map_async(check_for_issues, doing)

In [14]:
from iuvs.multitools import nb_progress_display

In [71]:
nb_progress_display(results, doing)

In [72]:
results.progress

1270

In [73]:
resultdf = pd.DataFrame(results.result)

In [77]:
resultdf.describe()

,n_unique_spabins,n_unique_spebins
count,1270,1270
mean,1,1
std,0,0
min,1,1
25%,1,1
50%,1,1
75%,1,1
max,1,1


In [74]:
for col in resultdf.columns:
    if col == 'fname': continue
    print(col)
    print(resultdf[col].value_counts(dropna=False))

kind
LIN    1270
Name: kind, dtype: int64
n_unique_spabins
1    1270
Name: n_unique_spabins, dtype: int64
n_unique_spebins
1    1270
Name: n_unique_spebins, dtype: int64


In [78]:
resultdf['phase'] = resultdf.fname.map(lambda x: io.ScienceFilename(x).phase)

In [90]:
subdf = resultdf[(resultdf.n_unique_spabins>1) | (resultdf.n_unique_spebins>1)]

In [91]:
subdf[subdf.kind=='LIN'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 5 columns):
fname               0 non-null object
kind                0 non-null object
n_unique_spabins    0 non-null int64
n_unique_spebins    0 non-null int64
phase               0 non-null object
dtypes: int64(2), object(3)
memory usage: 0.0+ bytes


## Scanning code

In [79]:
def myfilter(fname, d):
    from iuvs import exceptions, io
    okay = True
    try:
        l1a = io.L1AReader(fname)
    except exceptions.DimensionsError:
        d['error'] = 'dims'
        l1a = None
        okay = False
        return d, l1a, False
    if l1a.img_header['BIN_TBL'].startswith('NON LINEAR'):
        d['error'] = 'nonlinear'
        okay = False
    elif l1a.spatial_size != 7 or l1a.spectral_size != 256:
        d['error'] = 'binning'
        okay = False
    return d, l1a, okay

In [80]:
def get_primary_dark(fname):
    from iuvs import exceptions, io
    import numpy as np
    d = dict(fname=fname)
    d, l1a, okay = myfilter(fname, d)
    if okay is not True:
        return d
    if l1a.n_dims == 2:
        return l1a.primary_img_dn_s[np.newaxis,...]
    else:
        return l1a.primary_img_dn_s

In [81]:
from iuvs.multitools import *

In [82]:
from ipywidgets import HTML

In [83]:
container = []
errors = []
prog = IntProgress(min=0, max=len(df.p))
display(prog)
t = HTML()
display(t)
for i,fname in enumerate(df.p.iloc[:]):
    prog.value = i
    t.value = fname.name
    ret = get_primary_dark(fname)
    if isinstance(ret, dict):
        errors.append(ret)
    else:
        container.append(ret)

KeyboardInterrupt: 

In [30]:
periapse_darks = np.concatenate(container, axis=0)

In [31]:
periapse_darks.shape

(4030, 7, 256)

In [32]:
panel = pd.Panel(periapse_darks)

In [33]:
panel.to_hdf('/Users/klay6683/Dropbox/data/iuvs/periapse_muvdarks_v02_r01/periapse_darks.hdf', 'panel')

In [72]:
def create_summary_df(fname):
    from iuvs import exceptions, io
    import numpy as np
    d = dict(fname=fname)
    d, l1a, okay = myfilter(fname, d)
    if not okay:
        return d

    main_header = io.get_header_df(l1a.hdulist[0])
    integration = l1a.Integration

    if main_header.loc[0, 'NAXIS'] == 2:
        main_header.loc[0, 'NAXIS3'] = np.nan
        avgtuple = None
    elif main_header.loc[0, 'NAXIS'] == 3:
        avgtuple = (1,2)
    else:
        d['error'] = 'axes'
        return d
    lenint = len(integration)
    if lenint > 1:
        main_header = main_header.append([main_header]*(lenint-1), ignore_index=True)
    joined = pd.concat([integration, main_header], axis=1)
    for col in l1a.Observation.names[:-3]:
        val = l1a.Observation[col][0]
        if col == 'COLLECTION_ID':
            val = val[0]
        joined[col] = val
#     savepath = io.save_to_hdf(joined.sort_index(axis=1), fname, 'l1a_dark_scans')
    d['success'] = True
#     return d
    return joined

In [84]:
dview.push({'myfilter':myfilter})

<AsyncResult: finished>

In [85]:
create_summary_df(l1adarkfnames[0])

,CASE_TEMP,DET_TEMP,ET,FOV_DEG,LYA_CENTROID,MIRROR_DEG,MIRROR_DN,TIMESTAMP,UTC,BIN_TBL,BITPIX,BLANK,CAPTURE,EXTEND,FILENAME,INT_TIME,MCP_VOLT,MIR_DEG,NAXIS,NAXIS1,NAXIS2,N_FILL,OBS_ID,PROCESS,SIMPLE,SPA_OFS,SPA_SIZE,SPE_OFS,SPE_SIZE,XUV,NAXIS3,PRODUCT_ID,COLLECTION_ID,BUNDLE_ID,CODE_SVN_REVISION,ANC_SVN_REVISION,PRODUCT_CREATION_DATE,OBSERVATION_TYPE,MISSION_PHASE,TARGET_NAME,ORBIT_SEGMENT,ORBIT_NUMBER,SOLAR_LONGITUDE,GRATING_SELECT,KEYHOLE_SELECT,BIN_PATTERN_INDEX,CADENCE,DUTY_CYCLE,CHANNEL
0,15950,2576,4.669038e+08,17.29248,-1,8.64624,16087,4.669037e+08,2014/291 Oct 18 11:28:44.36433UTC,"LINEAR 7,8 CSS_S4",32,-1,2014/291 Oct 18 11:28:44.36433UTC,True,mvn_iuv_l1a_periapse-orbit00108-muvdark_201410...,4.2,-1.83,8.649,2,256,7,0,1,2015/205 Jul 24 12:25:52.00000UTC,True,96,115,0,4,MUV,NaN,mvn_iuv_l1a_periapse-orbit00108-muvdark_201410...,limb,raw,1141,1141,2015/205 Jul 24 12:25:52.00000UTC,CDS,PRIME,,0,108,216.553055,LOWRES,NEITHER,LINEAR CSS_S4,4.8,1,MUV


In [94]:
doing = l1adarkfnames
results = lview.map_async(create_summary_df, doing)

In [95]:
nb_progress_display(results, doing)

In [97]:
dfs = [df for df in results.result if type(df)!=dict]

In [98]:
results_df = pd.concat(dfs)

In [100]:
results_df.head()

,ANC_SVN_REVISION,BIN_PATTERN_INDEX,BIN_TBL,BITPIX,BLANK,BUNDLE_ID,CADENCE,CAPTURE,CASE_TEMP,CHANNEL,CODE_SVN_REVISION,COLLECTION_ID,DET_TEMP,DUTY_CYCLE,ET,EXTEND,FILENAME,FOV_DEG,GRATING_SELECT,INT_TIME,KEYHOLE_SELECT,LYA_CENTROID,MCP_VOLT,MIRROR_DEG,MIRROR_DN,MIR_DEG,MISSION_PHASE,NAXIS,NAXIS1,NAXIS2,NAXIS3,N_FILL,OBSERVATION_TYPE,OBS_ID,ORBIT_NUMBER,ORBIT_SEGMENT,PROCESS,PRODUCT_CREATION_DATE,PRODUCT_ID,SIMPLE,SOLAR_LONGITUDE,SPA_OFS,SPA_SIZE,SPE_OFS,SPE_SIZE,TARGET_NAME,TIMESTAMP,UTC,XUV
0,1141,LINEAR CSS_S4,"LINEAR 7,8 CSS_S4",32,-1,raw,4.8,2014/291 Oct 18 11:28:44.36433UTC,15950,MUV,1141,limb,2576,1,4.669038e+08,True,mvn_iuv_l1a_periapse-orbit00108-muvdark_201410...,17.292480,LOWRES,4.2,NEITHER,-1,-1.83,8.646240,16087,8.649,PRIME,2,256,7,NaN,0,CDS,1,108,0,2015/205 Jul 24 12:25:52.00000UTC,2015/205 Jul 24 12:25:52.00000UTC,mvn_iuv_l1a_periapse-orbit00108-muvdark_201410...,True,216.553055,96,115,0,4,,4.669037e+08,2014/291 Oct 18 11:28:44.36433UTC,MUV
0,1141,LINEAR CSS_S4,"LINEAR 7,8 CSS_S4",32,-1,raw,4.8,2014/291 Oct 18 11:50:24.86118UTC,15929,MUV,1141,limb,2570,1,4.669051e+08,True,mvn_iuv_l1a_periapse-orbit00108-muvdark_201410...,23.373413,LOWRES,4.2,NEITHER,-1,-1.83,11.686707,17194,11.689,PRIME,2,256,7,NaN,0,CDS,1,108,0,2015/205 Jul 24 12:26:24.00000UTC,2015/205 Jul 24 12:26:24.00000UTC,mvn_iuv_l1a_periapse-orbit00108-muvdark_201410...,True,216.562317,96,115,0,4,,4.669050e+08,2014/291 Oct 18 11:50:24.86118UTC,MUV
0,1141,LINEAR CSS_S4,"LINEAR 7,8 CSS_S4",32,-1,raw,4.8,2014/291 Oct 18 16:05:46.53431UTC,15950,MUV,1141,limb,2576,1,4.669204e+08,True,mvn_iuv_l1a_periapse-orbit00109-muvdark_201410...,17.292480,LOWRES,4.2,NEITHER,-1,-1.83,8.646240,16087,8.649,PRIME,2,256,7,NaN,0,CDS,1,109,0,2015/205 Jul 24 12:27:52.00000UTC,2015/205 Jul 24 12:27:52.00000UTC,mvn_iuv_l1a_periapse-orbit00109-muvdark_201410...,True,216.671570,96,115,0,4,,4.669203e+08,2014/291 Oct 18 16:05:46.53431UTC,MUV
0,1141,LINEAR CSS_S4,"LINEAR 7,8 CSS_S4",32,-1,raw,4.8,2014/291 Oct 18 16:27:26.52514UTC,15923,MUV,1141,limb,2570,1,4.669217e+08,True,mvn_iuv_l1a_periapse-orbit00109-muvdark_201410...,23.373413,LOWRES,4.2,NEITHER,-1,-1.83,11.686707,17194,11.689,PRIME,2,256,7,NaN,0,CDS,1,109,0,2015/205 Jul 24 12:28:24.00000UTC,2015/205 Jul 24 12:28:24.00000UTC,mvn_iuv_l1a_periapse-orbit00109-muvdark_201410...,True,216.680832,96,115,0,4,,4.669216e+08,2014/291 Oct 18 16:27:26.52514UTC,MUV
0,1141,LINEAR CSS_S4,"LINEAR 7,8 CSS_S4",32,-1,raw,4.8,2014/291 Oct 18 20:42:49.94587UTC,15949,MUV,1141,limb,2576,1,4.669370e+08,True,mvn_iuv_l1a_periapse-orbit00110-muvdark_201410...,17.292480,LOWRES,4.2,NEITHER,-1,-1.83,8.646240,16087,8.649,PRIME,2,256,7,NaN,0,CDS,1,110,0,2015/205 Jul 24 12:29:52.00000UTC,2015/205 Jul 24 12:29:52.00000UTC,mvn_iuv_l1a_periapse-orbit00110-muvdark_201410...,True,216.790115,96,115,0,4,,4.669370e+08,2014/291 Oct 18 20:42:49.94587UTC,MUV


In [62]:
results_df.loc[results_df.ANC_SVN_REVISION == '', 'ANC_SVN_REVISION'] = 0

In [63]:
results_df = results_df.convert_objects(convert_numeric=True)

In [64]:
results_df.to_hdf('/home/klay6683/output/l1a_dark_scans/results_df.h5', 'df')

## Merge temporary h5 files to database

In [277]:
import glob
h5fnames = glob.glob("/home/klay6683/output/l1a_dark_scans/*.h5")

In [278]:
len(h5fnames)

13107

In [279]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [280]:
dfs = []
for i,chunk in enumerate(chunker(h5fnames, 200)):
    print("Chunk {}".format(i))
    frames = []
    for fname in chunk:
        frames.append(pd.read_hdf(fname, 'df'))
    dfs.append(pd.concat(frames, ignore_index=True))

Chunk 0
Chunk 1
Chunk 2
Chunk 3
Chunk 4
Chunk 5
Chunk 6
Chunk 7
Chunk 8
Chunk 9
Chunk 10
Chunk 11
Chunk 12
Chunk 13
Chunk 14
Chunk 15
Chunk 16
Chunk 17
Chunk 18
Chunk 19
Chunk 20
Chunk 21
Chunk 22
Chunk 23
Chunk 24
Chunk 25
Chunk 26
Chunk 27
Chunk 28
Chunk 29
Chunk 30
Chunk 31
Chunk 32
Chunk 33
Chunk 34
Chunk 35
Chunk 36
Chunk 37
Chunk 38
Chunk 39
Chunk 40
Chunk 41
Chunk 42
Chunk 43
Chunk 44
Chunk 45
Chunk 46
Chunk 47
Chunk 48
Chunk 49
Chunk 50
Chunk 51
Chunk 52
Chunk 53
Chunk 54
Chunk 55
Chunk 56
Chunk 57
Chunk 58
Chunk 59
Chunk 60
Chunk 61
Chunk 62
Chunk 63
Chunk 64
Chunk 65


In [282]:
superdf = pd.concat(dfs, ignore_index=True)

In [283]:
superdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51306 entries, 0 to 51305
Data columns (total 52 columns):
ANC_SVN_REVISION         51306 non-null object
BIN_PATTERN_INDEX        51306 non-null object
BIN_TBL                  51306 non-null object
BITPIX                   51306 non-null int64
BLANK                    51306 non-null int64
BUNDLE_ID                51306 non-null object
CADENCE                  51306 non-null float64
CAPTURE                  51306 non-null object
CASE_TEMP                51306 non-null float64
CHANNEL                  51306 non-null object
CODE_SVN_REVISION        51306 non-null object
COLLECTION_ID            51306 non-null object
DET_TEMP                 51306 non-null float64
DUTY_CYCLE               51306 non-null float64
ET                       51306 non-null float64
EXTEND                   51306 non-null bool
FILENAME                 51306 non-null object
FOV_DEG                  51306 non-null float32
GRATING_SELECT           51306 non-null obj

In [284]:
from iuvs import calib

In [285]:
superdf.DET_TEMP = superdf.DET_TEMP.map(calib.convert_det_temp_to_C)
superdf.CASE_TEMP = superdf.CASE_TEMP.map(calib.convert_case_temp_to_C)

In [286]:
superdf.to_hdf('/home/klay6683/to_keep/l1a_dark_scan.h5','df')

In [287]:
from iuvs import meta

In [103]:
from iuvs import calib

In [104]:
results_df.DET_TEMP = results_df.DET_TEMP.map(calib.convert_det_temp_to_C)

In [105]:
results_df.CASE_TEMP = results_df.CASE_TEMP.map(calib.convert_case_temp_to_C)

In [107]:
results_df.to_hdf('/Users/klay6683/Dropbox/data/iuvs/periapse_muvdarks_v02_r01/results_df.hdf',
                 'df')